# 🎤 AudioToTextDiarazer: Расшифровка аудио и видео с диаризацией

Этот блокнот позволяет:
1. Загрузить аудио или видео файл
2. Извлечь из него аудио (если это видео)
3. Распознать речь с помощью Whisper
4. Выполнить диаризацию (определение говорящих)
5. Сохранить результаты в различных форматах


## 1. Клонирование репозитория и установка зависимостей


In [ ]:
# Клонирование репозитория
!git clone https://github.com/your-username/AudioToTextDiarazer.git
%cd AudioToTextDiarazer


## 2. Загрузка файла

Загрузите ваш аудио/видео файл через интерфейс Colab или подключите Google Drive


In [ ]:
# Вариант 1: Загрузка через интерфейс Colab
from google.colab import files

uploaded = files.upload()
input_filename = next(iter(uploaded.keys()))
print(f"Загружен файл: {input_filename}")

# Создаём директорию для входных файлов
!mkdir -p INPUT
!mv "{input_filename}" INPUT/input_file
input_path = "INPUT/input_file"


In [ ]:
# Вариант 2: Подключение к Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Создание директорий для хранения результатов
drive_output_dir = '/content/drive/MyDrive/audio_to_text_results'
!mkdir -p {drive_output_dir}
print(f"Результаты будут сохранены в: {drive_output_dir}")

# Если хотите использовать файл с Google Drive - раскомментируйте строку ниже
# input_path = '/content/drive/MyDrive/путь_к_файлу.mp4'  # Укажите путь к вашему файлу


## 3. Настройка параметров


In [ ]:
# Основные настройки
START_MIN = None      # Начало обработки (минуты), None = с начала файла
END_MIN = None        # Конец обработки (минуты), None = до конца файла
DIARIZE = False       # Включить диаризацию (определение говорящих)
NUM_SPEAKERS = 2      # Количество говорящих (для диаризации)
LANG = "ru"           # Язык распознавания

# Настройки модели (для продвинутых пользователей)
PREFER_GPU_ASR = True         # Использовать GPU, если доступен
FORCE_CPU_ASR = True          # Принудительно использовать CPU (стабильнее)
ASR_MODEL_GPU = "distil-large-v3" 
ASR_MODEL_CPU = "small"

# HuggingFace токен для PyAnnote (для лучшей диаризации)
# import os
# os.environ["HF_TOKEN"] = "ваш_токен_huggingface"  # Раскомментируйте и укажите свой токен


## 4. Запуск процесса распознавания и диаризации


In [ ]:
# Модификация скрипта для работы с загруженным файлом
with open('audio_to_text_diarizer.py', 'r') as f:
    script = f.read()

# Замена пути к входному файлу и других параметров
script = script.replace('INPUT_MEDIA   = "input.mp4"', f'INPUT_MEDIA   = "{input_path}"')
script = script.replace('START_MIN     = None', f'START_MIN     = {START_MIN}')
script = script.replace('END_MIN       = None', f'END_MIN       = {END_MIN}')
script = script.replace('DIARIZE       = False', f'DIARIZE       = {DIARIZE}')
script = script.replace('NUM_SPEAKERS  = 2', f'NUM_SPEAKERS  = {NUM_SPEAKERS}')
script = script.replace('LANG          = "ru"', f'LANG          = "{LANG}"')

# Запись временного скрипта
with open('temp_script.py', 'w') as f:
    f.write(script)

# Запуск скрипта
%run temp_script.py


## 5. Просмотр результатов


In [ ]:
import json
from IPython.display import display, HTML

# Загрузка результатов
try:
    with open('run_out/final/final.json', 'r', encoding='utf-8') as f:
        results = json.load(f)
    
    # Отображение сегментов с временными метками
    html = "<h3>Результаты распознавания</h3><table border=1>"
    html += "<tr><th>Время</th><th>Говорящий</th><th>Текст</th></tr>"
    
    for seg in results.get('segments', []):
        start = f"{int(seg['start']//60):02d}:{int(seg['start']%60):02d}"
        end = f"{int(seg['end']//60):02d}:{int(seg['end']%60):02d}"
        speaker = seg.get('speaker', '')
        text = seg.get('text', '').strip()
        html += f"<tr><td>{start}-{end}</td><td>{speaker}</td><td>{text}</td></tr>"
    
    html += "</table>"
    display(HTML(html))
except Exception as e:
    print(f"Ошибка при загрузке результатов: {e}")


## 6. Скачивание результатов


In [ ]:
from google.colab import files
import shutil
import os
from datetime import datetime

# Создаем уникальное имя для архива с результатами
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_name = f"audio_results_{timestamp}.zip"

# Архивирование результатов
!zip -r {archive_name} run_out/

# Сохранение результатов на Google Drive
if 'drive_output_dir' in globals():
    drive_archive = os.path.join(drive_output_dir, archive_name)
    shutil.copy(archive_name, drive_archive)
    print(f"Результаты сохранены на Google Drive: {drive_archive}")

# Скачивание архива локально
print("Скачивание результатов на ваш компьютер...")
files.download(archive_name)
